# Homework 1 (Due: February 27, 2023)

## 1. Fitting a model to data

1. Prepare an artificial data set composed of $t$ = \[0.0, 0.1, 0.2,..., 200\] and $y = 0.8e^{-c_1 t}+0.2 e^{-c_2 t}$ where $c_1$ = 0.1 and $c_2$ = 0.01. Then, use [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) to find the best fit of our "model" of the functional form $y = A \exp(-a t)$ where $A$ and $a$ are constant parameters.
2. Add gaussian noise ([`numpy.random.randn`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)) to the $y$ data and see how the model fitting is affected depending on the amplitude of the noise. Also, compare the results with a model with two exponential terms, $y = A_1 \exp(-a_1 t) + A_2 \exp(-a_2 t)$. How would you determine which model is better given the goodness of fit and number of parameters?
3. Load the data file `ipsc_acc_17711000.dat` (Use [`numpy.loadtxt`](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html)). Note that the first and second column are time in millisecond and a time series derived from experimental data, respectively) and try to fit the sum-of-exponentials models to this data. Choose your best model/fit and describe how you validate the result.

In [55]:
import numpy as np
import scipy.optimize as op
import plotly.express as px
import plotly.graph_objects as go


# 1.
t = np.arange(0, 200, 0.1)
c1 = 0.1
c2 = 0.01
y = 0.8 * np.exp(- c1 * t) + 0.2 * np.exp(- c2 * t)

def model(t, A, a):
    return A * np.exp(-a*t)

params, _ = op.curve_fit(model, t, y, [1]*2)
res = model(t, *params)


# 2.
y_noise = y + np.random.normal(0, 0.1, len(y))

def model2(t, A1, a1, A2, a2):
    return A1 * np.exp(- a1 * t) + A2 * np.exp(- a2 * t)

params_noise, _ = op.curve_fit(model, t, y_noise, [1]*2)
res_noise = model(t, *params_noise)

params2, _ = op.curve_fit(model2, t, y, [0.5]*4)
res2 = model2(t, *params2)
params2_noise, _ = op.curve_fit(model2, t, y_noise, [0.5]*4)
res2_noise = model2(t, *params2_noise)


fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=y,
                    mode='markers+lines',
                    name='artificial y'))
fig.add_trace(go.Scatter(x=t, y=y_noise,
                    mode='markers+lines',
                    name='artificial y with noise'))
fig.add_trace(go.Scatter(x=t, y=res, 
                    mode='lines',
                    name='model 1 fitted to y'))
fig.add_trace(go.Scatter(x=t, y=res_noise,
                    mode='lines',
                    name='model 1 fitted to y with noise'))
fig.add_trace(go.Scatter(x=t, y=res2,
                    mode='lines',
                    name='model 2 fitted to y'))
fig.add_trace(go.Scatter(x=t, y=res2_noise,
                    mode='lines',
                    name='model 2 fitted to y with noise'))

fig.show()
print(params2, params2_noise)

/var/folders/b2/rp63_y3x3l761csvpcgxn18h0000gp/T/ipykernel_3600/3307922662.py:24: RuntimeWarning:

overflow encountered in exp

/var/folders/b2/rp63_y3x3l761csvpcgxn18h0000gp/T/ipykernel_3600/3307922662.py:24: RuntimeWarning:

overflow encountered in multiply



[0.2  0.01 0.8  0.1 ] [0.20392674 0.00985996 0.79206266 0.09675666]


In [61]:
data = np.loadtxt("ipsc_acc_17711000.dat")
t_data = data[:, 0]
y_data = data[:, 1]

y_data_norm = y_data[len(y_data)//2:]

def model_n(t, *params):
    assert len(params) % 2 == 0, "Number of parameters must be even"
    n = len(params) // 2
    result = 0
    for i in range(n):
        A_i, a_i = params[2*i], params[2*i+1]
        result += A_i * np.exp(- a_i * t)
    return result

n_expos_lst = [1, 2, 3, 4]
res_data_lst = []
for n_expos in n_expos_lst:
    params_data, _ = op.curve_fit(model_n, t_data, y_data, [1]*2*n_expos)
    res_data_lst.append(model_n(t, *params_data))


fig = go.Figure()
fig.add_trace(go.Scatter(x=t_data, y=y_data,
                    mode='markers+lines',
                    name='data'))
for i in range(len(n_expos_lst)):
    fig.add_trace(go.Scatter(x=t_data, y=res_data_lst[i],
                        mode='lines',
                        name=f'{i+1} exponentials fitted to data'))

fig.show()


/var/folders/b2/rp63_y3x3l761csvpcgxn18h0000gp/T/ipykernel_3600/3565552774.py:13: RuntimeWarning:

overflow encountered in exp

/var/folders/b2/rp63_y3x3l761csvpcgxn18h0000gp/T/ipykernel_3600/3565552774.py:13: RuntimeWarning:

overflow encountered in multiply

/var/folders/b2/rp63_y3x3l761csvpcgxn18h0000gp/T/ipykernel_3600/3565552774.py:13: RuntimeWarning:

invalid value encountered in add

/var/folders/b2/rp63_y3x3l761csvpcgxn18h0000gp/T/ipykernel_3600/3565552774.py:13: RuntimeWarning:

overflow encountered in add



## 2. Inspecting membrane properties via current clamp

Following an example in our NEURON Tutorial 1, create an axon simulation
with the following specifications:

- Diameter (`diam`) = 1 um and length (`L`) = 1000 um,
- Capacitance per area (`cm`) = 1 uF/cm2,
- Passive mechanism (`pas`) with the conductance per area = 1×10$^{-4}$ S/cm2. (**Hint**: the conductance variable for this mechanism is `axon.g_pas`)
- A current clamp electrode (`ic`) near one end (`x`≈0) that injects 50 pA for a sufficiently long time (>200 ms).

1. What are the theoretical membrane time constant (τ) and space constant (λ)?
2. Run the simulation and measure the membrane time constant from the simulated data. How does the measured time constant compare with the theoretical value?
3. What is the electrotonic length of this axon?
4. **(Fig. 2.4)** By changing the length (`L`), plot the voltage attenuation curve in Fig. 2.4 in the textbook. Note that the unconnected cylinder ends in NEURON are sealed, and you will reproduce the "Sealed end" case. Discuss implications in membrane potential dynamics in short versus long dendrites.


## 3. Fig. 2.5 and 2.8B

Using the simulation in Quesion 2, plot the sealed end case of Fig. 2.5 in the textbook by following the steps below. 

1. **(I-V curve)** Try a few different values for the injected current from -50 pA to 50 pA, record the steady state membrane voltages, and plot the current vs membrane voltage graph. What is the input resistance?
2. **(Fig. 2.5)** Draw the cable length curve versus input resistance curve as Fig. 2.5 in the textbook. Please make sure that the cable has the sufficiently large `nseg` to get a quasi-smooth curve. Again, you should be able to get only one curves since cables in NEURON have sealed-ends.
3. **(Fig. 2.8B)** Set the length sufficiently long and plot how the voltage changes in time during current injection at different locations as in Fig. 2.8B.